Using repaired data to calculate new losses and accuracies for each client.

The new training data is taken from 'Data/train_data_repaired.csv'. This has been saved in the file 'data_repairing.ipynb'


In [2]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from sklearn.metrics import precision_score,recall_score, accuracy_score,confusion_matrix,f1_score
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import History

import pandas as pd



C:\Users\riash\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
C:\Users\riash\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
### Train/Test Data seperation

file_out = pd.read_csv('Data/adult_processed.csv')
file_out_repaired = pd.read_csv('Data/train_data_repaired.csv')
cols = []
cols_train = []
for i in list(file_out.columns):
    if  i != 'income':
        cols.append(i)
        
for i in list(file_out_repaired.columns):
    if  i != 'income':
        cols_train.append(i)

feature_set1 = pd.read_csv('Data/train_data_repaired.csv')
feature_set2 = pd.read_csv('Data/test.csv')

x = feature_set1[cols_train].copy().values
y = feature_set1[['income']].copy().values
        
sc = StandardScaler()
X_train = sc.fit_transform(x)
y_train = y

x2 = feature_set2[cols].copy().values
y2 = feature_set2[['income']].copy().values
        
X_test = sc.fit_transform(x2)
y_test = y2


# X_test.shape, y_test.shape 
if cols == cols_train:
    print("yes")
# diff = np.setdiff1d(cols_train,cols)
# diff

yes


In [4]:
num_clients = 10 # enter NUMBER OF CLIENTS

In [5]:
# gender split

dict_users = {i: np.array([]) for i in range(num_clients)} 
data_out = []

def create_hetero_clients( image_list, label_list, start_client = 0, num_clients=10, initial='clients'):
    
    selected_inds = []

    max_y = np.argmax(label_list, axis=-1)
    sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
    data = [(x,y) for _,y,x in sorted_zip]
    
    data_out = data
    
    num_shards, num_imgs = int(len(image_list)/30), 30
    idx_shard = [i for i in range(num_shards)]

    min_shard = 1
    max_shard = 60  #953/15 = 63.53
    
    random_shard_size = np.random.randint(min_shard, max_shard+1,
                                          size=(num_clients-start_client))
    random_shard_size = np.around(random_shard_size /
                                  sum(random_shard_size) * num_shards)
    random_shard_size = random_shard_size.astype(int)


    if sum(random_shard_size) > num_shards:
        
        for i in range(start_client, num_clients):
            # First assign each client 1 shard to ensure every client has
            # atleast one shard of data
            rand_set = set(np.random.choice(idx_shard, 1, replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

        random_shard_size = random_shard_size-1

        # Next, randomly assign the remaining shards
        for i in range(start_client, num_clients):
            if len(idx_shard) == 0:
                continue
#             print(random_shard_size)
            shard_size = random_shard_size[i-start_client]
            if shard_size > len(idx_shard):
                shard_size = len(idx_shard)
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

    else:

        for i in range(start_client, num_clients):
#             print(random_shard_size)
            shard_size = random_shard_size[i-start_client]
#             shard_size = random_shard_size[int(i/len(random_shard_size)) - 1]
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[i]) == 0):
                    dict_users[i] = data[ind1: ind2]
                else:
                    dict_users[i] = np.concatenate(
                    (dict_users[i],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])

        if len(idx_shard) > 0:
            # Add the leftover shards to the client with minimum images:
            shard_size = len(idx_shard)
            # Add the remaining shard to the client with lowest data
            k = min(dict_users, key=lambda x: len(dict_users.get(x)))
            rand_set = set(np.random.choice(idx_shard, shard_size,
                                            replace=False))
            idx_shard = list(set(idx_shard) - rand_set)
            for rand in rand_set:
                ind1 = rand*num_imgs
                ind2 = (rand+1)*num_imgs
                if(len(dict_users[k]) == 0):
                    dict_users[k] = data[ind1: ind2]
                else:
                    dict_users[k] = np.concatenate(
                    (dict_users[k],data[ind1: ind2]),
                    axis=0)
                selected_inds.extend([[ind1, ind2]])
                
                
    return dict_users, selected_inds, data_out



In [6]:
# dict_users = {i: np.array([]) for i in range(4)}

def create_client_iid(image_list, label_list, client_num):    
    max_y = np.argmax(label_list, axis=-1)
    sorted_zip = sorted(zip(max_y, label_list, image_list), key=lambda x: x[0])
    data = [(x,y) for _,y,x in sorted_zip]
    
    dict_users[client_num] = data
    
    return dict_users

In [7]:
def initializer_income():
    
    x_feats = feature_set1[cols_train].copy().values
    y_feats = feature_set1[['income']].copy().values

    X_train_new = sc.fit_transform(x_feats)
    y_train_new = y_feats

    
    return [X_train_new, y_train_new]

In [8]:
def get_hetero_clients_gender(train_sets):
    
    data = list(zip(train_sets[0], train_sets[1]))
    random.shuffle(data)
    
#     print(type(train_sets), type(train_sets[0]),type(list(list(zip(*data))[0]) ), type(data))
    
    train_sets[0] = list(list(zip(*data))[0])
    train_sets[1] = list(list(zip(*data))[1])
    
    
    # NON_IID
    clients, inds, data_out1 = create_hetero_clients(train_sets[0], train_sets[1], start_client = 0, num_clients=10, initial='client') 
    
    

# #     IID
#     clients = create_client_iid(train_sets[0], train_sets[1], 0)
    
#     clients2 = create_client_iid(train_sets[2], train_sets[3], 1)
#     clients = {**clients, **clients2}
    
#     clients3 = create_client_iid(train_sets[4], train_sets[5], 2)
#     clients = {**clients, **clients3}
    
#     clients4 = create_client_iid(train_sets[6], train_sets[7], 3)
#     clients = {**clients, **clients4}
    

    
    return clients


In [9]:
# take bs = 128 for 5 clients and 10 rounds
def batch_data(data_shard, bs=64):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    data = []
    label = []
    for x in data_shard:
        data.append(x[0])
        label.append(x[1])
    #seperate shard into data and labels lists
#     data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
#     print( label[0])
    return dataset.shuffle(len(label)).batch(bs)

In [15]:

#process and batch the training data for each client
def batch_clients(clients):
    clients_batched = dict()
    for (client_name, data) in clients.items():
#         print("data ",len(data))
        clients_batched[client_name] = batch_data(data)#non-IID..126
#         clients_batched[client_name] = batch_data(data,1) #IID
    

    #process and batch the test set  
    test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))
    
#     test_batched
    return clients_batched, test_batched


In [11]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes=2 , learning_rate = 0.001, metric = "accuracy"):

        model = Sequential()
        
        model.add(Dense(128, input_shape = (shape,)))
#         model.add(Dense(128, Activation("relu")))
#         model.add(Dense(64, Activation("relu")))
#         model.add(Dense(32, Activation("relu")))
#         model.add(Dense(1))
        
        model.add(Dense(128, Activation("tanh")))
        model.add(Dense(64, Activation("tanh")))
        model.add(Dense(32, Activation("tanh")))
        model.add(Dense(1,Activation('sigmoid')))
        


        return model

In [12]:
lr = 0.005
comms_round = 10
loss=tf.keras.losses.BinaryCrossentropy(from_logits = False)

metrics = ['binary_accuracy']

optimizer = SGD(learning_rate=lr, 
                decay=lr / comms_round, 
                momentum=0.5
               )     



In [13]:


def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
#     print(len(scaled_weight_list))
    for grad_list_tuple in zip(*scaled_weight_list):
#         print(len(grad_list_tuple))
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    
#     cce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
#     cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    logits = model.predict(X_test)

    score = global_model.evaluate(X_test, y_test, verbose=0)
    acc = score[1] ; loss = score[0]
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))

    return acc, loss

In [16]:
#### #initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(X_train.shape[1] ,classes=2)
global_model.compile(optimizer=optimizer, loss=loss, metrics=metrics) 

client_names = {i for i in range(num_clients)}

global_loss_list = []
global_freq_list = []
global_acc_list = []
global_di_list = []
client_loss = {i: np.array([]) for i in range(len(client_names))}
client_accuracy = {i: np.array([]) for i in range(len(client_names))}
client_frequency = {i: np.array([]) for i in range(len(client_names))}
client_scores = {i: np.array({}) for i in range(len(client_names))}
client_DI = {i: np.array([]) for i in range(len(client_names))}

train_sets = initializer_income()
# train_sets = initializer_income_race_gender()

# print(len(train_sets))
#commence global training loop
for comm_round in range(comms_round):
    epoch_freq = 0
#     clients = create_clients(X_train, y_train, num_clients=15, initial='client')
#     clients = get_hetero_clients()
    clients = get_hetero_clients_gender(list(train_sets) )
#     clients = get_hetero_clients_gender_race(list(train_sets) )
#     print("client0: ", clients[0])

    clients_batched, test_batched = batch_clients(clients)
#     print("client batched 0: ", clients_batched[1])
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
#     print(client_names)
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(X_train.shape[1],classes=2)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        history = local_model.fit(clients_batched[client], epochs=1, verbose=1)
                                  
        nn_preds = local_model.predict(X_test)
        nn_preds = (nn_preds > 0.5)

        nn_precision =precision_score(y_test, nn_preds)
        nn_recall = recall_score(y_test, nn_preds) # = sensitivity
        nn_accuracy = accuracy_score(y_test, nn_preds)
        nn_f1 = f1_score(y_test, nn_preds)
        nn_specificity = recall_score(y_test, nn_preds, pos_label = 0)


#         print("Precision = {}".format(nn_precision))
#         print("Recall = {}".format(nn_recall))
#         print("Accuracy = {}".format(nn_accuracy))
#         print("f1 = {}".format(nn_f1))
#         print("Specificity = {}".format(nn_specificity))
        
        DI = (1-nn_specificity)/nn_recall
        
#         get client acc, loss
#         print(client)
        if(len(client_loss[client])== 0):
            client_loss[client] = [history.history['loss'][0]]
            client_accuracy[client] = [history.history['binary_accuracy'][0]]
            client_frequency[client] = [len(clients_batched[client])]
            client_scores[client] = [ {"precision":nn_precision, "recall": nn_recall, "accuracy": nn_accuracy, "f1": nn_f1,
                                 "specificity": nn_specificity,
                                    "sensitivity": nn_recall}]
            client_DI[client] = [DI]
        
        else:
            client_loss[client] = np.append(client_loss[client], (history.history['loss'][0]))
            client_frequency[client] = np.append(client_frequency[client], len(clients_batched[client]))
            client_accuracy[client] = np.append(client_accuracy[client], (history.history['binary_accuracy'][0]))
            client_scores[client] = np.append(client_scores[client], {"precision":nn_precision, "recall": nn_recall, "accuracy": nn_accuracy, "f1": nn_f1,
                                 "specificity": nn_specificity,
                                    "sensitivity": nn_recall})
            client_DI[client] = np.append(client_DI[client],DI)
        
        epoch_freq += len(clients_batched[client])
        
#         print("client", client, "\nloss -->" ,client_loss[client], "\nfreq->", client_frequency[client], "\naccuracy->", client_accuracy[client], 
#              "\nscores: ", client_scores[client])
        print("client", client, "\nDI -->" ,client_DI[client])
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)
    
    global_freq_list.append(epoch_freq)
    epoch_freq = 0
    

#     test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        global_loss_list.append(global_loss)
        global_acc_list.append(global_acc)
        global_preds = global_model.predict(X_test)
        global_preds = (global_preds > 0.5)
        
        nn_recall2 = recall_score(y_test, global_preds)
        nn_specificity2 = recall_score(y_test, global_preds, pos_label = 0)
        
        DI2 = (1-nn_specificity2)/nn_recall2
        
        global_di_list.append(DI2)
        
        print("global_loss_list: ", global_loss_list )
        print("global_acc_list: ", global_acc_list )
        print("global_freq_list: ", global_freq_list)
        print("global_di_list: ", global_di_list)
        



<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


240/240 [==============================] - 0s 1ms/step - loss: 0.4711 - binary_accuracy: 0.7822
client 0 
DI --> [0.1688877258048966]
195/195 [==============================] - 0s 1ms/step - loss: 0.5017 - binary_accuracy: 0.7566
client 9 
DI --> [0.1944519612739815]
286/286 [==============================] - 0s 1ms/step - loss: 0.4683 - binary_accuracy: 0.7841
client 2 
DI --> [0.17236396455887693]
160/160 [==============================] - 0s 1ms/step - loss: 0.5285 - binary_accuracy: 0.7435
client 4 
DI --> [0.22159586770488007]
195/195 [==============================] - 0s 1ms/step - loss: 0.5189 - binary_accuracy: 0.7508
client 3 
DI --> [0.20639566975246554]
317/317 [==============================] - 0s 1ms/step - loss: 0.4799 - binary_accuracy: 0.7743
client 8 
DI --> [0.16979755046236472]
256/256 [==============================] - 0s 1ms/step - loss: 0.5042 - binary_accuracy: 0.7627
client 1 
DI --> [0.18280735746568533]
209/209 [==============================] - 0s 1ms/step - 

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


444/444 [==============================] - 1s 2ms/step - loss: 0.3774 - binary_accuracy: 0.8316
client 6 
DI --> [0.18536078 0.15113115]
307/307 [==============================] - 0s 1ms/step - loss: 0.3884 - binary_accuracy: 0.8290
client 1 
DI --> [0.18280736 0.15269396]
337/337 [==============================] - 1s 2ms/step - loss: 0.3878 - binary_accuracy: 0.8252
client 8 
DI --> [0.16979755 0.15548713]
299/299 [==============================] - 0s 2ms/step - loss: 0.3887 - binary_accuracy: 0.8275
client 3 
DI --> [0.20639567 0.15588213]
257/257 [==============================] - 0s 1ms/step - loss: 0.3955 - binary_accuracy: 0.8223
client 4 
DI --> [0.22159587 0.15991252]
226/226 [==============================] - 0s 1ms/step - loss: 0.3964 - binary_accuracy: 0.8245
client 9 
DI --> [0.19445196 0.1627382 ]
290/290 [==============================] - 1s 2ms/step - loss: 0.3888 - binary_accuracy: 0.8301
client 2 
DI --> [0.17236396 0.1621848 ]
219/219 [==============================] 

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


...

559/559 [==============================] - 1s 2ms/step - loss: 0.3470 - binary_accuracy: 0.8420
client 7 
DI --> [0.21861554 0.16359869 0.15047636 0.14239667 0.14175956 0.13772435
 0.13761881 0.13441479]
686/686 [==============================] - 1s 2ms/step - loss: 0.3462 - binary_accuracy: 0.8402
client 5 
DI --> [0.20242245 0.1595276  0.14980851 0.14568717 0.14180835 0.14089839
 0.14007112 0.1389143 ]
564/564 [==============================] - 1s 2ms/step - loss: 0.3457 - binary_accuracy: 0.8420
client 9 
DI --> [0.19445196 0.1627382  0.15197971 0.14404299 0.1421006  0.14079189
 0.14004256 0.13770062]
646/646 [==============================] - 1s 1ms/step - loss: 0.3423 - binary_accuracy: 0.8428
client 3 
DI --> [0.20639567 0.15588213 0.14988459 0.14594012 0.13984023 0.13806413
 0.13612242 0.13598414]
861/861 [==============================] - 1s 2ms/step - loss: 0.3417 - binary_accuracy: 0.8441
client 6 
DI --> [0.18536078 0.15113115 0.14403548 0.14219541 0.13985969 0.13640284
 0.1

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


981/981 [==============================] - 2s 2ms/step - loss: 0.3416 - binary_accuracy: 0.8439
client 6 
DI --> [0.18536078 0.15113115 0.14403548 0.14219541 0.13985969 0.13640284
 0.13633112 0.13441479 0.13511604]
608/608 [==============================] - 1s 1ms/step - loss: 0.3427 - binary_accuracy: 0.8434
client 0 
DI --> [0.16888773 0.1584348  0.14540802 0.14160249 0.13949557 0.13723683
 0.13660826 0.13570582 0.13532296]
590/590 [==============================] - 1s 1ms/step - loss: 0.3452 - binary_accuracy: 0.8418
client 7 
DI --> [0.21861554 0.16359869 0.15047636 0.14239667 0.14175956 0.13772435
 0.13761881 0.13441479 0.13479153]
717/717 [==============================] - 1s 1ms/step - loss: 0.3386 - binary_accuracy: 0.8443
client 2 
DI --> [0.17236396 0.1621848  0.15013541 0.14517893 0.14156718 0.14055163
 0.13782148 0.1373303  0.13739571]
703/703 [==============================] - 1s 1ms/step - loss: 0.3422 - binary_accuracy: 0.8442
client 1 
DI --> [0.18280736 0.15269396 0.14

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


738/738 [==============================] - 1s 2ms/step - loss: 0.3418 - binary_accuracy: 0.8432
client 3 
DI --> [0.20639567 0.15588213 0.14988459 0.14594012 0.13984023 0.13806413
 0.13612242 0.13598414 0.1359138  0.13521942]
699/699 [==============================] - 1s 2ms/step - loss: 0.3425 - binary_accuracy: 0.8428
client 0 
DI --> [0.16888773 0.1584348  0.14540802 0.14160249 0.13949557 0.13723683
 0.13660826 0.13570582 0.13532296 0.13539505]
798/798 [==============================] - 1s 2ms/step - loss: 0.3393 - binary_accuracy: 0.8444
client 8 
DI --> [0.16979755 0.15548713 0.14856112 0.14357459 0.14148777 0.13892865
 0.13792677 0.13643431 0.1356054  0.13553791]
678/678 [==============================] - 1s 2ms/step - loss: 0.3442 - binary_accuracy: 0.8421
client 7 
DI --> [0.21861554 0.16359869 0.15047636 0.14239667 0.14175956 0.13772435
 0.13761881 0.13441479 0.13479153 0.13622697]
826/826 [==============================] - 1s 2ms/step - loss: 0.3367 - binary_accuracy: 0.8451


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

pdfs = []
for i in list(client_accuracy.keys()):
    print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i], "acc", client_accuracy[i])
    
#     norm
#     mean, std = stats.norm.fit(client_loss[i])
#     pdf =  stats.norm.pdf(client_loss[i], mean, std)

#     df_mean = np.mean(client_frequency[i])
#     df_std = np.std(client_frequency[i])
#     pdf = stats.norm.pdf(client_frequency[i], df_mean, df_std)
    
#     lognorm
#     shape, loc, scale = stats.lognorm.fit(client_loss[i])
#     pdf = stats.lognorm.pdf(client_loss[i], shape, loc, scale)

#     beta
#     beta_params = stats.beta.fit(client_loss[i])
#     pdf = stats.beta.pdf(client_loss[i], beta_params[0], beta_params[1], beta_params[2], beta_params[3])

#     burr
#     burr_params = stats.burr.fit(client_loss[i])
#     pdf = stats.burr.pdf(client_loss[i], burr_params[0], burr_params[1], burr_params[2], burr_params[3])

    
#  gamma
    shape, loc, scale = stats.gamma.fit(client_loss[i])
    pdf = stats.gamma.cdf(client_loss[i], shape, loc=loc, scale=scale)
    
    pdfs.append(pdf)
    sns.set_style("whitegrid")

    ax = sns.displot( x = pdf, kind = "kde", height=10, aspect=2,
                linewidth = 5 )
    plt.ylabel("LOSS")
#     plt.xlabel("Loss")
    plt.xlabel("CDF")
    
    plt.show()

In [ ]:
# !pip install --force-reinstall --no-deps matplotlib==3.4.3
# !pip install matplotlib

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats 

for i in list(client_accuracy.keys()):
    print("\nclient_",i)
#     spec = []
#     sens = []
#     for j in range(len( client_scores[i])):
#         spec.append(client_scores[i][j]["specificity"])
#         sens.append(client_scores[i][j]["sensitivity"])
#     print("\nspecificity: ", spec, "\n\nsensitivity: ", sens)
        
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc: ", client_accuracy[i], 
#           "\nspecificity: ", client_scores[i][9]["specificity"], "sensitivity: ", client_scores[i][9]["sensitivity"], 
#          "\nDI: ", client_DI[i])
    print("DI: ", client_DI[i])
#      print("client_",i, 
#           "\nspecificity: ", client_scores[i][9]["specificity"], "sensitivity: ", client_scores[i][9]["sensitivity"])
print("\nglobal_DI: ", global_di_list)


client_ 0
DI:  [0.16888773 0.1584348  0.14540802 0.14160249 0.13949557 0.13723683
 0.13660826 0.13570582 0.13532296 0.13539505]

client_ 1
DI:  [0.18280736 0.15269396 0.1469135  0.14077845 0.14089059 0.13813669
 0.1354866  0.136157   0.13580749 0.13496949]

client_ 2
DI:  [0.17236396 0.1621848  0.15013541 0.14517893 0.14156718 0.14055163
 0.13782148 0.1373303  0.13739571 0.13646851]

client_ 3
DI:  [0.20639567 0.15588213 0.14988459 0.14594012 0.13984023 0.13806413
 0.13612242 0.13598414 0.1359138  0.13521942]

client_ 4
DI:  [0.22159587 0.15991252 0.14629058 0.14146376 0.13763391 0.13755329
 0.13493257 0.13580973 0.13490268 0.13451803]

client_ 5
DI:  [0.20242245 0.1595276  0.14980851 0.14568717 0.14180835 0.14089839
 0.14007112 0.1389143  0.1386602  0.13773262]

client_ 6
DI:  [0.18536078 0.15113115 0.14403548 0.14219541 0.13985969 0.13640284
 0.13633112 0.13441479 0.13511604 0.13448982]

client_ 7
DI:  [0.21861554 0.16359869 0.15047636 0.14239667 0.14175956 0.13772435
 0.13761881 0.

In [17]:
for i in list(client_accuracy.keys()):
    print("\nclient_",i)
#     spec = []
#     sens = []
#     for j in range(len( client_scores[i])):
#         spec.append(client_scores[i][j]["specificity"])
#         sens.append(client_scores[i][j]["sensitivity"])
#     print("\nspecificity: ", spec, "\n\nsensitivity: ", sens)
        
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc: ", client_accuracy[i], 
#           "\nspecificity: ", client_scores[i][9]["specificity"], "sensitivity: ", client_scores[i][9]["sensitivity"], 
#          "\nDI: ", client_DI[i])
    print("freq: ", client_frequency[i])
#      print("client_",i, 
#           "\nspecificity: ", client_scores[i][9]["specificity"], "sensitivity: ", client_scores[i][9]["sensitivity"])
print("\nglobal_freq: ", global_freq_list)


client_ 0
freq:  [240 310 359 368 386 450 532 581 608 699]

client_ 1
freq:  [256 307 378 485 520 597 628 640 703 728]

client_ 2
freq:  [286 290 404 452 519 558 631 698 717 826]

client_ 3
freq:  [195 299 310 416 508 530 602 646 685 738]

client_ 4
freq:  [160 257 262 385 472 566 658 732 834 881]

client_ 5
freq:  [262 368 448 457 505 518 586 686 780 826]

client_ 6
freq:  [327 444 446 503 593 643 733 861 981 987]

client_ 7
freq:  [209 219 251 357 377 486 529 559 590 678]

client_ 8
freq:  [317 337 459 464 537 659 668 751 753 798]

client_ 9
freq:  [195 226 349 392 470 491 540 564 678 776]

global_freq:  [2447, 3057, 3666, 4279, 4887, 5498, 6107, 6718, 7329, 7937]


In [ ]:
print("_GAMMA_")

cdfs_loss_gamma = []
cdfs_acc_gamma = []
cdfs_freq_gamma = []
for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

    shape, loc, scale = stats.gamma.fit(client_loss[i])
    cdf_loss_gamma =  stats.gamma.cdf(client_loss[i], shape, loc, scale)
    cdfs_loss_gamma.append(cdf_loss_gamma)

    shape, loc, scale= stats.gamma.fit(client_accuracy[i])
    cdf_acc_gamma = stats.gamma.cdf(client_accuracy[i],shape, loc, scale)
    cdfs_acc_gamma.append(cdf_acc_gamma)

    shape, loc, scale  = stats.gamma.fit(client_frequency[i])
    cdf_freq_gamma = stats.gamma.cdf(client_frequency[i],shape, loc, scale )
    cdfs_freq_gamma.append(cdf_freq_gamma)


print("ACCURACY")
for i in range(len(cdfs_acc_gamma)):
    for j in range(i+1,len(cdfs_acc_gamma)):
        print(i," and ", j , stats.wasserstein_distance(cdfs_acc_gamma[i], cdfs_acc_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_acc_list)
cdf_global_acc_gamma =  stats.gamma.cdf(global_acc_list, shape, loc, scale)

for i in range(len(cdfs_acc_gamma)):
    print(i," and global: " , stats.wasserstein_distance(cdfs_acc_gamma[i], cdf_global_acc_gamma))

print("LOSS")
for i in range(len(cdfs_loss_gamma)):
    for j in range(i+1,len(cdfs_loss_gamma)):
        print(i," and ", j , stats.wasserstein_distance(cdfs_loss_gamma[i], cdfs_loss_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_loss_list)
cdf_global_loss_gamma =  stats.gamma.cdf(global_loss_list, shape, loc, scale)

for i in range(len(cdfs_loss_gamma)):
    print(i," and global: " , stats.wasserstein_distance(cdfs_loss_gamma[i], cdf_global_loss_gamma))

print("FREQUENCY")
for i in range(len(cdfs_freq_gamma)):
    for j in range(i+1,len(cdfs_freq_gamma)):
        print(i," and ", j , stats.wasserstein_distance(cdfs_freq_gamma[i], cdfs_freq_gamma[j]))


shape, loc, scale = stats.gamma.fit(global_freq_list)
cdf_global_freq_gamma =  stats.gamma.cdf(global_freq_list,shape, loc, scale)

for i in range(len(cdfs_freq_gamma)):
    print(i," and global: " , stats.wasserstein_distance(cdfs_freq_gamma[i], cdf_global_freq_gamma))

In [ ]:

# for i in range(len(pdfs_loss_gamma)):
#     print(i," and global: " , stats.wasserstein_distance(pdfs_loss_gamma[i], pdf_global_loss_gamma))
#     M = ot.dist(pdfs_loss_gamma[i], pdf_global_loss_gamma, metric='euclidean')
#     W = ot.emd2(pdfs_loss_gamma[i], pdf_global_loss_gamma, M)
#     print("actual ", W)

In [ ]:
print("_NORM_")

pdfs_loss_norm = []
pdfs_acc_norm = []
pdfs_freq_norm = []
for i in list(client_accuracy.keys()):
#     print("client_",i, " loss: ", client_loss[i],"freq: ", client_frequency[i], "acc", client_accuracy[i])
    
    mean, std = stats.norm.fit(client_loss[i])
    pdf_loss_norm =  stats.norm.pdf(client_loss[i], mean, std)
    pdfs_loss_norm.append(pdf_loss_norm)

    mean, std = stats.norm.fit(client_accuracy[i])
    pdf_acc_norm = stats.norm.pdf(client_accuracy[i],mean, std )
    pdfs_acc_norm.append(pdf_acc_norm)
    
    mean, std  = stats.norm.fit(client_frequency[i])
    pdf_freq_norm = stats.norm.pdf(client_frequency[i], mean, std )
    pdfs_freq_norm.append(pdf_freq_norm)
    
    
print("ACCURACY")
for i in range(len(pdfs_acc_norm)):
    for j in range(i+1,len(pdfs_acc_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc_norm[i], pdfs_acc_norm[j]))
        
        
mean, std = stats.norm.fit(global_acc_list)
pdf_global_acc_norm =  stats.norm.pdf(global_acc_list, mean, std)

for i in range(len(pdfs_acc_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc_norm[i], pdf_global_acc_norm))
    
print("LOSS")
for i in range(len(pdfs_loss_norm)):
    for j in range(i+1,len(pdfs_loss_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss_norm[i], pdfs_loss_norm[j]))
        
        
mean, std = stats.norm.fit(global_loss_list)
pdf_global_loss_norm =  stats.norm.pdf(global_loss_list, mean, std)

for i in range(len(pdfs_loss_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss_norm[i], pdf_global_loss_norm))
    
print("FREQUENCY")
for i in range(len(pdfs_freq_norm)):
    for j in range(i+1,len(pdfs_freq_norm)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq_norm[i], pdfs_freq_norm[j]))
        
        
mean, std = stats.norm.fit(global_freq_list)
pdf_global_freq_norm =  stats.norm.pdf(global_freq_list, mean, std)

for i in range(len(pdfs_freq_norm)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq_norm[i], pdf_global_freq_norm))


In [ ]:
print("_LOGNORM_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):
#     print("client_",i, "\nloss: ", client_loss[i],"\nfreq: ", client_frequency[i], "\nacc", client_accuracy[i])

    shape, loc, scale = stats.lognorm.fit(client_loss[i])
    pdf_loss =  stats.lognorm.pdf(client_loss[i], shape, loc, scale)
    pdfs_loss.append(pdf_loss)

    shape, loc, scale= stats.lognorm.fit(client_accuracy[i])
    pdf_acc = stats.lognorm.pdf(client_accuracy[i],shape, loc, scale)
    pdfs_acc.append(pdf_acc)

    shape, loc, scale  = stats.lognorm.fit(client_frequency[i])
    pdf_freq = stats.lognorm.pdf(client_frequency[i],shape, loc, scale )
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


shape, loc, scale = stats.lognorm.fit(global_acc_list)
pdf_global_acc =  stats.lognorm.pdf(global_acc_list, shape, loc, scale)

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


shape, loc, scale = stats.lognorm.fit(global_loss_list)
pdf_global_loss =  stats.lognorm.pdf(global_loss_list, shape, loc, scale)

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


shape, loc, scale = stats.lognorm.fit(global_freq_list)
pdf_global_freq =  stats.lognorm.pdf(global_freq_list,shape, loc, scale)

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

In [ ]:
print("_BETA_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):

    beta_params = stats.beta.fit(client_loss[i])
    pdf_loss =  stats.beta.pdf(client_loss[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_loss.append(pdf_loss)

    beta_params = stats.beta.fit(client_accuracy[i])
    pdf_acc = stats.beta.pdf(client_accuracy[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_acc.append(pdf_acc)

    beta_params  = stats.beta.fit(client_frequency[i])
    pdf_freq = stats.beta.pdf(client_frequency[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


beta_params = stats.beta.fit(global_acc_list)
pdf_global_acc =  stats.beta.pdf(global_acc_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


beta_params = stats.beta.fit(global_loss_list)
pdf_global_loss =  stats.beta.pdf(global_loss_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


beta_params = stats.beta.fit(global_freq_list)
pdf_global_freq =  stats.beta.pdf(global_freq_list,beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

In [ ]:
print("_BURR_")

pdfs_loss = []
pdfs_acc = []
pdfs_freq = []
for i in list(client_accuracy.keys()):

    params = stats.burr.fit(client_loss[i])
    pdf_loss =  stats.burr.pdf(client_loss[i],params[0], params[1], params[2], params[3])
    pdfs_loss.append(pdf_loss)

    beta_params = stats.burr.fit(client_accuracy[i])
    pdf_acc = stats.burr.pdf(client_accuracy[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_acc.append(pdf_acc)

    beta_params  = stats.burr.fit(client_frequency[i])
    pdf_freq = stats.burr.pdf(client_frequency[i],beta_params[0], beta_params[1], beta_params[2], beta_params[3])
    pdfs_freq.append(pdf_freq)


print("ACCURACY")
for i in range(len(pdfs_acc)):
    for j in range(i+1,len(pdfs_acc)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_acc[i], pdfs_acc[j]))


beta_params = stats.burr.fit(global_acc_list)
pdf_global_acc =  stats.burr.pdf(global_acc_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_acc)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_acc[i], pdf_global_acc))

print("LOSS")
for i in range(len(pdfs_loss)):
    for j in range(i+1,len(pdfs_loss)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_loss[i], pdfs_loss[j]))


beta_params = stats.burr.fit(global_loss_list)
pdf_global_loss =  stats.burr.pdf(global_loss_list, beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_loss)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_loss[i], pdf_global_loss))

print("FREQUENCY")
for i in range(len(pdfs_freq)):
    for j in range(i+1,len(pdfs_freq)):
        print(i," and ", j , stats.wasserstein_distance(pdfs_freq[i], pdfs_freq[j]))


beta_params = stats.burr.fit(global_freq_list)
pdf_global_freq =  stats.burr.pdf(global_freq_list,beta_params[0], beta_params[1], beta_params[2], beta_params[3])

for i in range(len(pdfs_freq)):
    print(i," and global: " , stats.wasserstein_distance(pdfs_freq[i], pdf_global_freq))

In [ ]:
for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
    print("client_",i, "loss: ", client_loss[i])

In [ ]:
for i in list(client_loss.keys()):
#     if(i== len(list(client_loss.keys()))-1):
#         continue
    print("client_",i, "freq: ", client_frequency[i])

In [ ]:
for i in range(len(pdfs)):
    for j in range(i+1,len(pdfs)):
        print(i," and ", j , stats.wasserstein_distance(pdfs[i], pdfs[j]))

In [ ]:
# # gamma
shape, loc, scale = stats.gamma.fit(global_freq_list)
pdf_global = stats.gamma.pdf(global_freq_list, shape, loc=loc, scale=scale)

# df_mean = np.mean(global_loss_list)
# df_std = np.std(global_loss_list)
# pdf_global = stats.norm.pdf(global_loss_list, df_mean, df_std)

# mean, std = stats.norm.fit(global_freq_list)
# pdf_global =  stats.norm.pdf(global_freq_list, mean, std)

# lognorm
# shape, loc, scale = stats.lognorm.fit(global_loss_list)
# pdf_global = stats.lognorm.pdf(global_loss_list, shape, loc, scale)

# global_acc_list
# global_freq_list

# burr
# burr_params = stats.beta.fit(global_freq_list)
# pdf_global = stats.beta.pdf(global_freq_list, burr_params[0], burr_params[1], burr_params[2], burr_params[3])

for i in range(len(pdfs)):
    print(i," and global: " , stats.wasserstein_distance(pdfs[i], pdf_global))

In [ ]:
!pip install fitter

In [ ]:
for i in range(len(pdfs)):
    for j in range(i+1,len(pdfs)):
        print(i," and ", j , stats.wasserstein_distance(pdfs[i], pdfs[j]))

In [ ]:
shape, loc, scale = stats.gamma.fit(global_loss_list)
pdf_global = stats.gamma.pdf(global_loss_list, shape, loc=loc, scale=scale)

for i in range(len(pdfs)):
    print(i," and global: " , stats.wasserstein_distance(pdfs[i], pdf_global))

In [ ]:
score = global_model.evaluate(X_test, y_test, verbose=0)
score

In [ ]:
nn_preds = global_model.predict(X_test)
nn_preds = (nn_preds > 0.5)

nn_precision =precision_score(y_test, nn_preds)
nn_recall = recall_score(y_test, nn_preds)
nn_accuracy = accuracy_score(y_test, nn_preds)
nn_f1 = f1_score(y_test, nn_preds)


print("Precision = {}".format(nn_precision))
print("Recall = {}".format(nn_recall))
print("Accuracy = {}".format(nn_accuracy))
print("f1 = {}".format(nn_f1))


In [ ]:
arr = nn_preds > 0.5

unique, counts = np.unique(arr, return_counts=True)

np.asarray((unique, counts)).T